In [7]:
!pip install cmake dlib

  Using cached dlib-19.24.0.tar.gz (3.2 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for dlib: filename=dlib-19.24.0-cp38-cp38-macosx_10_15_x86_64.whl size=3538547 sha256=4ab606002b7b25d647bc1bfb573d63c216411c4594934b4bbf3217f81b05ebca
  Stored in directory: /Users/shrey/Library/Caches/pip/wheels/4c/d8/2d/a83b10e7bf10cd7d8bef36bf4dcd15b0c9ebf98f990bc984dd
Successfully built dlib


In [9]:
!pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


In [1]:
import face_recognition
import cv2
import numpy as np
import glob
import os
import logging
import datetime

In [2]:
IMAGES_PATH = '/Users/shrey/Desktop/XYZZZZZ/image'
CROPPED_IMAGES_PATH = '/Users/shrey/Desktop/XYZZZZZ/cropeed'
CAMERA_DEVICE_ID = 0
MAX_DISTANCE = 0.6

In [3]:
def get_face_embeddings_from_image(image, convert_to_rgb=False):
    """
    Take a raw image and run both the face detection and face embedding model on it
    """
    
    # Convert from BGR to RGB if needed
    if convert_to_rgb:
        image = image[:, :, ::-1]
        
    # Run the face detection model to find face locations
    face_locations = face_recognition.face_locations(image)
    
    # Run the embedding model to get face embeddings for the supplied locations
    face_embeddings = face_recognition.face_encodings(image, face_locations)
    
    return face_locations, face_embeddings

In [4]:
def setup_database():
    """
    Load reference images and create a database of their face encodings
    """
    database = {}
    
    for filename in glob.glob(os.path.join('/Users/shrey/Desktop/XYZZZZZ/image','*.jpeg')):
        # Load image
        image_rgb = face_recognition.load_image_file(filename)
        
        # Use the name in the filename as the identity key
        identity = os.path.splitext(os.path.basename(filename))[0]
        
        # Get the face encoding and link it to the identity
        locations, encodings = get_face_embeddings_from_image(image_rgb)
        database[identity] = encodings[0]
    
    return database

In [5]:
def paint_detected_face_on_image(frame, location, name=None):
    """
    Paint a rectangle around the face and write the name
    """
    # unpack the coordinates from the location tuple
    top, right, bottom, left = location
    
    if name is None:
        name = 'Unknown'
        color = (0, 0, 255)  # red for unrecognized face
    else:
        color = (0, 128, 0)  # dark green for recognized face
        
    # Draw a box around the face
    cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
    
    # Draw a label with a name below the face
    cv2.rectangle(frame, (left, bottom + 15), (right, bottom), color, cv2.FILLED)
    cv2.putText(frame, name, (left + 6, bottom + 12), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

In [6]:
def click_event(event, x, y, flags, params):
    """
    Crop an image based on the clicked detected face
    """

    # event is triggered with a mouse click
    if event == cv2.EVENT_LBUTTONUP:
        for location in face_locations:

            # unpack the coordinates from the location tuple
            top, right, bottom, left = location
            if (top < y < bottom) and  (left < x < right):
                frame_copy = np.copy(frame)
                roi = frame_copy[top:bottom, left:right]

                # give a unique name for the cropped image
                currentDT = datetime.datetime.now()
                cropped_name = os.path.join('/Users/shrey/Desktop/XYZZZZZ/cropeed', loc_name_dict[location] + '_' + str(currentDT) + '.jpeg')

                # save the cropped image
                cv2.imwrite(cropped_name, roi)

                # show the cropped image
                crop = cv2.imread(cropped_name)
                cv2.imshow('cropped image', crop)

                # re-run the run_face_recognition function
                run_face_recognition(database)


In [7]:
def run_face_recognition(database):
    """
    Start the face recognition via webcam
    """
    
    global face_locations
    global frame
    global name
    global loc_name_dict

    # Open a connection to the camera
    video_capture = cv2.VideoCapture(CAMERA_DEVICE_ID)
    
    # The face_recognition library uses keys and values of your database separately
    known_face_encodings = list(database.values())
    known_face_names = list(database.keys())

    
    while video_capture.isOpened():
        
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
        
        if not ret:
            logging.error("Could not read frame from camera. Stopping video capture.")
            break
        
        # Run detection and embedding models
        face_locations, face_encodings = get_face_embeddings_from_image(frame, convert_to_rgb=True)

        # Build a dictionary to pair the location with the name of the detected face 
        loc_name_dict = dict()
        
        # Loop through each face in this frame of video and see if there's a match
        for location, face_encoding in zip(face_locations, face_encodings):
            
            # get the distances from this encoding to those of all reference images
            distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            
            # select the closest match (smallest distance) if it's below the threshold value
            if np.any(distances <= MAX_DISTANCE):
                best_match_idx = np.argmin(distances)
                name = known_face_names[best_match_idx]
            else:
                name = None
            
            # Pair the location with the name of the detected face inside a dictionary
            loc_name_dict[location] = name
            
            # show recognition info on the image
            paint_detected_face_on_image(frame, location, name)
            
        # Display the resulting image
        cv2.imshow('frame', frame)

        # Crop image if triggered by a mouse click
        cv2.setMouseCallback('frame', click_event)
    
        # Hit 'q' on the keyboard to stop the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # When everything done, release the capture
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
database = setup_database()
run_face_recognition(database)

1   HIToolbox                           0x00007ff81dade726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff81dade2b8 _ZN15MenuBarInstance14EnableAutoShowEv + 52
3   HIToolbox                           0x00007ff81da82908 SetMenuBarObscured + 408
4   HIToolbox                           0x00007ff81da824ca _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 164
5   HIToolbox                           0x00007ff81da7c996 _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 252
6   HIToolbox                           0x00007ff81da44bd2 _NotifyEventLoopObservers + 153
7   HIToolbox                           0x00007ff81da7c3e6 AcquireEventFromQueue + 494
8   HIToolbox                           0x00007ff81da6b3ec ReceiveNextEventCommon + 285
9   HIToolbox                           0x00007ff81da6b2b3 _BlockUntilNextEventMatchingListInModeWithFilter + 70
10  AppKit                              0x00007ff81726e2f3 _DPS